# Exploratory Data Analysis and Predictive Modelling of Local Atmospheric Pollutants

###### Kaggle Tabular Playground Series - July 2021

The [given dataset](https://www.kaggle.com/c/tabular-playground-series-jul-2021) includes dates, temperatures, humidities (relative and absolute), and data from sensors.

The goal of the project is to accurately predict the amounts of three target variables (pollutants);
target_carbon_monoxide, target_benzene, and target_nitrogen_oxides.

An additional personal goal is to determine the factors which effect each of the target variables the most.

I've also seen others use Plotly to visualise the data and findings. As I don't have much experience with Plotly in
Python, I'll be using it here also, as an opportunity to learn.

In [115]:
# import libraries

import os
import datetime as dt  # processing of dates and times

import numpy as np  # calculations and functions
print('numpy version : {}'.format(np.__version__))
import pandas as pd  # data processing / wrangling
print('pandas version : {}'.format(pd.__version__))

import matplotlib
import matplotlib.pyplot as plt  # data visualisation
from IPython.core.display import display
%matplotlib inline
print('matplotlib version : {}'.format(matplotlib.__version__))

import plotly
import plotly.express as px
import plotly.graph_objects as go
print('plotly version : {}'.format(plotly.__version__))

import sklearn
print('sklearn version : {}'.format(sklearn.__version__))

numpy version : 1.20.2
pandas version : 1.2.5
matplotlib version : 3.3.4
plotly version : 4.14.3
sklearn version : 0.24.2


## Data Processing

### Reading and Inspecting Datasets

First, we import both the training and testing datasets. We also create a new dataframe containing the data from both
the training and testing datasets, minus the target data. This makes the following data processing steps easier.

In [104]:
# path to directory containing input files
input_dir = os.getcwd() + os.sep + 'input'

# read input data files
df_train = pd.read_csv(input_dir + os.sep + 'train.csv')
df_test = pd.read_csv(input_dir + os.sep + 'test.csv')

# remove target columns from train data and append to test data
target_cols = ['target_carbon_monoxide',
               'target_benzene',
               'target_nitrogen_oxides']
df_all = df_train.drop(columns=target_cols).append(df_test)

Next, let's take a look at the data to get to grips with what we're working with.

In [105]:
# training dataframe
print('Training data shape : ', df_train.shape)
print(df_train.info())
display(df_train.head(5))

# testing dataframe
print('Testing data shape : ', df_test.shape)
print(df_test.info())
display(df_test.head())

Training data shape :  (7111, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date_time               7111 non-null   object 
 1   deg_C                   7111 non-null   float64
 2   relative_humidity       7111 non-null   float64
 3   absolute_humidity       7111 non-null   float64
 4   sensor_1                7111 non-null   float64
 5   sensor_2                7111 non-null   float64
 6   sensor_3                7111 non-null   float64
 7   sensor_4                7111 non-null   float64
 8   sensor_5                7111 non-null   float64
 9   target_carbon_monoxide  7111 non-null   float64
 10  target_benzene          7111 non-null   float64
 11  target_nitrogen_oxides  7111 non-null   float64
dtypes: float64(11), object(1)
memory usage: 666.8+ KB
None


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


Testing data shape :  (2247, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date_time          2247 non-null   object 
 1   deg_C              2247 non-null   float64
 2   relative_humidity  2247 non-null   float64
 3   absolute_humidity  2247 non-null   float64
 4   sensor_1           2247 non-null   float64
 5   sensor_2           2247 non-null   float64
 6   sensor_3           2247 non-null   float64
 7   sensor_4           2247 non-null   float64
 8   sensor_5           2247 non-null   float64
dtypes: float64(8), object(1)
memory usage: 158.1+ KB
None


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


There's a few things we should note about this data:
- Both datasets are relatively small, this may influence the type of model we choose.
- Certain input data (features) could be expanded. For example, the date can be used to add features such as season or
day of the week.
- There are no missing values (None or Null) in either set.

### Date and Time Processing

As usual, the dates and times listed in the date_time column could do with some work.

First, we'll inspect the dates and times and see if we can spot any patterns.

In [113]:
print('Training data:')
print(df_train.date_time)
print('')
print('Testing data:')
print(df_test.date_time)

Training data:
0       2010-03-10 18:00:00
1       2010-03-10 19:00:00
2       2010-03-10 20:00:00
3       2010-03-10 21:00:00
4       2010-03-10 22:00:00
               ...         
7106    2010-12-31 20:00:00
7107    2010-12-31 21:00:00
7108    2010-12-31 22:00:00
7109    2010-12-31 23:00:00
7110    2011-01-01 00:00:00
Name: date_time, Length: 7111, dtype: object

Testing data:
0       2011-01-01 00:00:00
1       2011-01-01 01:00:00
2       2011-01-01 02:00:00
3       2011-01-01 03:00:00
4       2011-01-01 04:00:00
               ...         
2242    2011-04-04 10:00:00
2243    2011-04-04 11:00:00
2244    2011-04-04 12:00:00
2245    2011-04-04 13:00:00
2246    2011-04-04 14:00:00
Name: date_time, Length: 2247, dtype: object


So it looks like we have a sample (row) for each hour, from 2010-03-10 6:00pm to 2011-01-01 12:00am for the training
data and from 2011-01-01 12:00am to 2011-04-04 2pm for the testing data.

Let's confirm that's the case by calculating the expected number of samples if we have 1 sample every hour, and
comparing this value to the actual number of samples.

In [107]:
# datetime format : (year, month, day, hour, minute, second)
train_initial_date = dt.datetime(2010, 3, 10, 18, 0, 0)
train_final_date = dt.datetime(2011, 1, 1, 0, 0, 0)
test_final_date = dt.datetime(2011, 4, 4, 14, 0, 0)

# difference between datetimes
SECONDS_PER_HOUR = 3600
train_timedelta = train_final_date - train_initial_date
train_hours = int(train_timedelta.total_seconds()/SECONDS_PER_HOUR)+1
test_timedelta = test_final_date - train_final_date
test_hours = int(test_timedelta.total_seconds()/SECONDS_PER_HOUR)+1

# get number of samples in datasets
train_num_samples = len(df_train)
test_num_samples = len(df_test)


# check if expected samples == actual samples
print('Training data:')
if train_hours == train_num_samples:
    print('1 sample every hour. No duplicates.')
else:
    print('Expected does not match actual number of samples.')
print('')
print('Testing data:')
if test_hours == test_num_samples:
    print('1 sample every hour. No duplicates.')
else:
    print('Expected does not match actual number of samples.')

Training data:
1 sample every hour. No duplicates.

Testing data:
1 sample every hour. No duplicates.


In validating that we have 1 sample every hour for both datasets, we discovered that the final date and time in the
training dataset is the same as the first date and time in the testing dataset.

Are these two samples identical?

In [108]:
display(df_train.iloc[-1:])
display(df_test.iloc[:1])

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
7110,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1,1.4,4.1,186.5


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1


Yes! These samples are identical. This is something to be aware of when we work with the concatenated df_all dataframe.

Now let's work on converting the date_time feature to the following features:
- Year
- Month
- Date
- Hour
- Minute
- Second
- Season
- Day of the week
- Time of day
- Weekend?
- Public holiday?

We'll start by breaking up date_time into its constituent parts.

In [109]:
# retain the intact date from the date_time values for ordering purposes
df_all['date'] = df_all.date_time.str.split(' ', expand=True)[0]

# convert values of date_time to datetime objects
df_all['date_time'] = pd.to_datetime(df_all['date_time'])
df_all['year'] = df_all['date_time'].dt.year
df_all['month'] = df_all['date_time'].dt.month
df_all['day'] = df_all['date_time'].dt.day
df_all['hour'] = df_all['date_time'].dt.hour
df_all['minute'] = df_all['date_time'].dt.minute
df_all['second'] = df_all['date_time'].dt.second

All of that was accomplished using the datetime library!

Now let's expand this feature even more, by adding some additional features from the list above.

The day of the week can also be found using the datetime library. From the day of the week we can determine whether the
date is on a weekday or a weekend.

The season is simply a function of the month. Similarly, the time of day is a function of the hour.

In [110]:
# create day of the week column
df_all['day_of_week'] = df_all['date_time'].dt.dayofweek

# create weekend column
df_all['weekend'] = 0
df_all.loc[df_all['day_of_week'].isin([5, 6, 7]), 'weekend'] = 1

# create season column (0 : winter, 1 : spring, 2 : summer, 3 : autumn)
df_all['season'] = 0
df_all.loc[df_all['month'].isin([3, 4, 5]), 'season'] = 1
df_all.loc[df_all['month'].isin([6, 7, 8]), 'season'] = 2
df_all.loc[df_all['month'].isin([9, 10, 11]), 'season'] = 3

# create time of day column (0 : morning, 1: afternoon, 2: evening, 3: night)
df_all['time_of_day'] = 0
df_all.loc[df_all['hour'].isin(range(12, 18)), 'season'] = 1
df_all.loc[df_all['hour'].isin(range(18, 23)), 'season'] = 2
df_all.loc[df_all['hour'].isin(range(23, 6)), 'season'] = 3

Whether the date falls on a public holiday is a little more complex, and involves importing data from
[here](https://www.kaggle.com/donnetew/us-holiday-dates-2004-2021).

In [111]:
# read csv file to pandas dataframe
df_holidays = pd.read_csv(input_dir + os.sep + 'us_holiday_dates_2004-2021.csv')

# inspect data
display(df_holidays.head())

# create public_holiday column on df_all
df_all['public_holiday'] = 0
df_all.loc[df_all['date'].isin(df_holidays.Date), 'public_holiday'] = 1

# remove the original date_time column
df_all = df_all.drop(columns='date_time')

,Date,Holiday,WeekDay,Month,Day,Year
0,2004-07-04,4th of July,Sunday,7,4,2004
1,2005-07-04,4th of July,Monday,7,4,2005
2,2006-07-04,4th of July,Tuesday,7,4,2006
3,2007-07-04,4th of July,Wednesday,7,4,2007
4,2008-07-04,4th of July,Friday,7,4,2008


Done! Here is the dataframe with the expanded date and time features:

In [112]:
display(df_all.head(3))

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,date,year,month,day,hour,minute,second,day_of_week,weekend,season,time_of_day,public_holiday
0,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2010-03-10,2010,3,10,18,0,0,2,0,2,0,0
1,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2010-03-10,2010,3,10,19,0,0,2,0,2,0,0
2,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2010-03-10,2010,3,10,20,0,0,2,0,2,0,0


### Additional Feature Expansion

Are there any other features we can extract from the current data set? Let's try to understand the other feature columns
by learning a bit about [humidity](https://www.engineersedge.com/thermodynamics/humidity.htm) and
[dew points](https://en.wikipedia.org/wiki/Dew_point).

**Absolute Humidity**<br>
*The total amount of water vapour present in a given volume of air, not taking temperature into consideration.*

**Relative Humidity**<br>
*The ratio of the partial pressure of water vapour in the mixture to the saturated vapour pressure of water at a given
temperature.*

**Dew Point**<br>
*The temperature at which air becomes saturated with water vapour and below which water vapour condenses to form liquid
(dew).*

We can use the relative humidities, and the temperatures, to *approximate* the
[dew point](https://bmcnoldy.rsmas.miami.edu/Humidity.html) using the following formula:

$$T_d = 243.04\times\frac{\log\left(\frac{RH}{100}\right)+\frac{17.625\times T}{243.04+T}}{17.625-\log\left(\frac{RH}{100}\right)-\frac{17.625\times T}{243.04+T}}$$

where $$T_d$$ is the dew point, $$RH$$ is the relative humidity, and $$T$$ is the temperature.

In [ ]:
# create dew point column
df_all['dew_point'] = 243.04*(np.log(df_all['relative_humidity']/100)+((17.625*df_all['deg_C'])/
                      (243.04+df_all['deg_C'])))/(17.625-np.log(df_all['relative_humidity']/100)-
                      ((17.625*df_all['deg_C'])/(243.04+df_all['deg_C'])))